In [1]:
#all imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx
import pandas as pd
from tqdm import tqdm

In [2]:
nodes = pd.read_csv("youtube_raw.csv").drop("id.1",axis = 1)

#eliminate recommendations
nodes = nodes.iloc[:,0:9]

In [3]:
#transform every column in list
for i in nodes.columns:
    globals()[i]=nodes[i].values.tolist()
    

In [4]:
#print all variables that are now available as lists
nodes.columns

Index(['id', 'uploader', 'age', 'category', 'length', 'views', 'rate',
       'ratings', 'comments'],
      dtype='object')

# Define features

In [ ]:
def Random_walk_features(G, source, target):
    
    #Random walk
    radnom_walk = 

    return [radnom_walk,
           ]

# Compute features

In [ ]:
#read train set and train graph
train_edges = pd.read_csv("train_set.csv", index_col =0).values.tolist()
G_train = nx.read_edgelist("train.edgelist", delimiter=',')

#read test set and test graph
test_edges = pd.read_csv("test_set.csv", index_col =0).values.tolist()
G_test = nx.read_edgelist("test.edgelist", delimiter=',')